In [3]:
import pandas as pd
from math import sqrt

In [4]:
laptop = pd.read_csv('data/Laptop.csv')

In [5]:
# print banyaknya data di setiap dataset
print('Laptop dataset has {} data'.format(len(laptop)))

Laptop dataset has 85 data


In [6]:
# print banyaknya data duplikat dalam dataset laptop
print('Laptop dataset has {} duplicate data'.format(laptop.duplicated().sum()))

Laptop dataset has 4 duplicate data


In [7]:
# print laptop "DELL XPS 9365"
laptop[laptop['Name'] == 'DELL XPS 9365']

,URL,ImageURLs,Name,Processor,Screen,InternalMemory,TotalStorage,Weight,BatteryCells,PointingDevice
35,https://www.laptoparena.net/dell/hybrid-(2-in-...,https://www.laptoparena.net/images/DELL_XPS_13...,DELL XPS 9365,1.3,13.3,8,256,1.24,4,Touchpad


In [8]:
laptop[['Name', 'Processor', 'Screen', 'InternalMemory', 'TotalStorage', 'Weight', 'BatteryCells', 'PointingDevice']]

,Name,Processor,Screen,InternalMemory,TotalStorage,Weight,BatteryCells,PointingDevice
0,Alienware 15 R3,2.8,15.6,16,1256,3.49,4,Touchpad
1,Alienware 17 R4,2.8,17.3,16,1256,4.42,6,Touchpad
2,DELL G3 3579,2.2,15.6,8,1128,2.35,4,Touchpad
3,DELL G3 3579,2.2,15.6,16,1128,2.35,4,Touchpad
4,DELL G3 3779,2.3,17.3,8,1128,2.81,4,Touchpad
...,...,...,...,...,...,...,...,...
80,Toshiba Satellite Pro Satellite Pro A50-C-204,2.3,15.6,4,500,2.10,4,Touchpad
81,Toshiba Tecra A40-C-1MR,2.3,15.6,8,256,2.20,4,Touchpad
82,Toshiba Tecra A50-D-10M,2.5,15.6,8,256,2.00,4,Touchpad
83,Toshiba Tecra A50-D-11D,2.7,15.6,8,256,2.00,4,Touchpad


In [9]:
# laptop[laptop['Name'] == 'Toshiba Tecra A50-D-10M']
print(laptop[laptop['Name'] == 'Alienware 15 R3']['PointingDevice'].values[0])

Touchpad


In [10]:
features = {
    'Processor': 'Processor',
    'Screen': 'Screen',
    'InternalMemory': 'InternalMemory',
    'TotalStorage': 'TotalStorage',
    'Weight': 'Weight',
    'BatteryCells': 'BatteryCells',
}

In [11]:
def euclideanDistance(data):
    return sqrt(sum([i ** 2 for i in data]))

In [12]:
# print nilai min dan max dari setiap features
for key, value in features.items():
    print('Min {} is {}'.format(value, laptop[key].min()))
    print('Max {} is {}'.format(value, laptop[key].max()))

Min Processor is 1.3
Max Processor is 3.1
Min Screen is 12.5
Max Screen is 17.3
Min InternalMemory is 4
Max InternalMemory is 32
Min TotalStorage is 128
Max TotalStorage is 2256
Min Weight is 1.2
Max Weight is 4.42
Min BatteryCells is 3
Max BatteryCells is 6


In [13]:
# Topsis Algorithm for laptop recommendation based on the 'Processor', 'Screen', 'InternalMemory', 'TotalStorage', 'Weight', 'BatteryCells'
weight = [1, 1, 1, 1, 1, 1, 1]
# impact = [0, 0, 0, 0, 0, 0]
impact = [1, 1, 1, 1, 1, 1, 1]

# Step 1: Normalisasi data
for key, value in features.items():
    laptop[value + 'Normalized'] = laptop[key] / euclideanDistance(laptop[key])
    # Step 2: Hitung weighted normalized decision matrix
    laptop[value + 'Normalized'] *= weight[list(features.keys()).index(key)]

# Step 3: Determine the ideal and negative-ideal solution
ideal = {}
negativeIdeal = {}
for key, value in features.items():
    if impact[list(features.keys()).index(key)] == 1:
        ideal[value] = laptop[value + 'Normalized'].max()
        negativeIdeal[value] = laptop[value + 'Normalized'].min()
    elif impact[list(features.keys()).index(key)] == 0:
        ideal[value] = laptop[value + 'Normalized'].min()
        negativeIdeal[value] = laptop[value + 'Normalized'].max()

# Step 4: Calculate the separation measures from the sum of all features
for key, value in features.items():
    laptop[value + 'SeparationMeasureIdeal'] = pow(laptop[value + 'Normalized'] - ideal[value], 2)
    laptop[value + 'SeparationMeasureNegativeIdeal'] = pow(laptop[value + 'Normalized'] - negativeIdeal[value], 2)

# Sum separation measures for all features
laptop['SeparationMeasureIdeal'] = laptop[[f'{value}SeparationMeasureIdeal' for value in features.values()]].sum(axis=1)
laptop['SeparationMeasureNegativeIdeal'] = laptop[[f'{value}SeparationMeasureNegativeIdeal' for value in features.values()]].sum(axis=1)

# Take the square root of the sum
laptop['SeparationMeasureIdeal'] = laptop['SeparationMeasureIdeal'].apply(sqrt)
laptop['SeparationMeasureNegativeIdeal'] = laptop['SeparationMeasureNegativeIdeal'].apply(sqrt)

# Step 5: Calculate the relative closeness to the ideal solution
laptop['totalRelativeCloseness'] = laptop['SeparationMeasureNegativeIdeal'] / (laptop['SeparationMeasureNegativeIdeal'] + laptop['SeparationMeasureIdeal'])

# Step 6: Rank the relative closeness and print top 10
laptop = laptop.sort_values(by=['totalRelativeCloseness'], ascending=False)

# Step 7: Print 5 recommended laptop, sorted by totalRelativeCloseness
laptop[['Name', 'totalRelativeCloseness']].head(5)

,Name,totalRelativeCloseness
42,DELL XPS 9570,0.735071
5,DELL G3 3779,0.639739
37,DELL XPS 9570,0.589304
43,DELL XPS 9570,0.579351
1,Alienware 17 R4,0.574360


In [14]:
# get two random recomendations from laptop
laptop1 = laptop[laptop['Name'] == 'DELL XPS 9570']
laptop2 = laptop.sample(n=1)
# calculate the sum of (negativeIdeal[value] - i) from every features for every laptop
for index, row in laptop.iterrows():
    for key, value in features.items():
        laptop.at[index, value + 'SeparationMeasureNegativeIdeal'] = sqrt(sum([(row[value + 'Normalized'] - negativeIdeal[value]) ** 2]))

laptop['SeparationMeasureNegativeIdeal'] = laptop[['ProcessorSeparationMeasureNegativeIdeal', 'ScreenSeparationMeasureNegativeIdeal', 'InternalMemorySeparationMeasureNegativeIdeal', 'TotalStorageSeparationMeasureNegativeIdeal', 'WeightSeparationMeasureNegativeIdeal', 'BatteryCellsSeparationMeasureNegativeIdeal']].sum(axis=1)

laptop['SeparationMeasureNegativeIdeal']

42    0.686143
5     0.556783
37    0.557646
43    0.525376
1     0.581121
        ...   
78    0.140927
38    0.144142
73    0.121157
35    0.083543
23    0.083542
Name: SeparationMeasureNegativeIdeal, Length: 85, dtype: float64

In [ ]:
print(weight)

[1, 1, 1, 1, 1, 1, 1]


In [ ]:
laptop1

,URL,ImageURLs,Breadcrumb 1,Breadcrumb 2,Breadcrumb 3,Name,Processor,Screen,InternalMemory,TotalStorage,...,InternalMemorySeparationMeasureNegativeIdeal,TotalStorageSeparationMeasureIdeal,TotalStorageSeparationMeasureNegativeIdeal,WeightSeparationMeasureIdeal,WeightSeparationMeasureNegativeIdeal,BatteryCellsSeparationMeasureIdeal,BatteryCellsSeparationMeasureNegativeIdeal,SeparationMeasureIdeal,SeparationMeasureNegativeIdeal,totalRelativeCloseness
42,https://www.laptoparena.net/product/22286,/images/tmb/DELL_XPS_9570_9570-0354_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.9,15.6,32,2000,...,0.062262,0.001082,0.057863,0.015839,0.000831,0.000000,0.005149,0.130953,0.363341,0.735071
37,https://www.laptoparena.net/product/22279,/images/tmb/DELL_XPS_9570_9570-1693_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.9,15.6,32,1000,...,0.062262,0.026047,0.012555,0.015839,0.000831,0.000000,0.005149,0.205217,0.294465,0.589304
43,https://www.laptoparena.net/product/22287,/images/tmb/DELL_XPS_9570_9570-0347_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,32,1000,...,0.062262,0.026047,0.012555,0.015839,0.000831,0.000000,0.005149,0.209166,0.288080,0.579351
45,https://www.laptoparena.net/product/28347,/images/tmb/DELL_XPS_9570_9570-0286_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.3,15.6,8,1128,...,0.001271,0.021009,0.016511,0.015839,0.000831,0.000000,0.005149,0.289990,0.162355,0.358919
39,https://www.laptoparena.net/product/22283,/images/tmb/DELL_XPS_9570_9570-0385_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,16,512,...,0.011436,0.050220,0.002435,0.015839,0.000831,0.000000,0.005149,0.297075,0.148472,0.333235
46,https://www.laptoparena.net/product/46579,/images/tmb/DELL_XPS_9570_9570-3634_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,16,512,...,0.011436,0.050220,0.002435,0.015839,0.000831,0.000000,0.005149,0.297075,0.148472,0.333235
40,https://www.laptoparena.net/product/22284,/images/tmb/DELL_XPS_9570_9570-0378_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,16,512,...,0.011436,0.050220,0.002435,0.015839,0.000831,0.000000,0.005149,0.297075,0.148472,0.333235
41,https://www.laptoparena.net/product/22285,/images/tmb/DELL_XPS_9570_9570-0361_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.3,15.6,8,1128,...,0.001271,0.021009,0.016511,0.015839,0.000831,0.005149,0.000000,0.298735,0.145638,0.327739
44,https://www.laptoparena.net/product/28339,/images/tmb/DELL_XPS_9570_9570-3450_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.3,15.6,8,1128,...,0.001271,0.021009,0.016511,0.015839,0.000831,0.005149,0.000000,0.298735,0.145638,0.327739
38,https://www.laptoparena.net/product/22281,/images/tmb/DELL_XPS_9570_9570-0309_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,8,256,...,0.001271,0.066046,0.000271,0.015839,0.000831,0.005149,0.000000,0.366935,0.067571,0.155512


In [ ]:
laptop2

,URL,ImageURLs,Breadcrumb 1,Breadcrumb 2,Breadcrumb 3,Name,Processor,Screen,InternalMemory,TotalStorage,...,InternalMemorySeparationMeasureNegativeIdeal,TotalStorageSeparationMeasureIdeal,TotalStorageSeparationMeasureNegativeIdeal,WeightSeparationMeasureIdeal,WeightSeparationMeasureNegativeIdeal,BatteryCellsSeparationMeasureIdeal,BatteryCellsSeparationMeasureNegativeIdeal,SeparationMeasureIdeal,SeparationMeasureNegativeIdeal,totalRelativeCloseness
46,https://www.laptoparena.net/product/46579,/images/tmb/DELL_XPS_9570_9570-3634_image_1.jp...,DELL,XPS,15,DELL XPS 9570,2.2,15.6,16,512,...,0.011436,0.05022,0.002435,0.015839,0.000831,0.0,0.005149,0.297075,0.148472,0.333235
